## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Reconquista,AR,2022-03-14 03:36:51,-29.1500,-59.6500,60.04,73,0,4.59,clear sky
1,1,Atuona,PF,2022-03-14 03:36:51,-9.8000,-139.0333,79.03,74,100,14.94,overcast clouds
2,2,Mahebourg,MU,2022-03-14 03:36:52,-20.4081,57.7000,81.25,89,75,8.05,broken clouds
3,3,East London,ZA,2022-03-14 03:33:54,-33.0153,27.9116,63.54,94,75,17.27,light rain
4,4,Itarema,BR,2022-03-14 03:36:52,-2.9248,-39.9167,77.27,85,98,5.66,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]



In [6]:
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Atuona,PF,2022-03-14 03:36:51,-9.8000,-139.0333,79.03,74,100,14.94,overcast clouds
2,2,Mahebourg,MU,2022-03-14 03:36:52,-20.4081,57.7000,81.25,89,75,8.05,broken clouds
4,4,Itarema,BR,2022-03-14 03:36:52,-2.9248,-39.9167,77.27,85,98,5.66,overcast clouds
7,7,Busselton,AU,2022-03-14 03:36:53,-33.6500,115.3333,78.85,54,64,6.76,broken clouds
9,9,Puerto Ayora,EC,2022-03-14 03:36:53,-0.7393,-90.3518,76.55,97,38,1.01,scattered clouds
11,11,Kudahuvadhoo,MV,2022-03-14 03:36:54,2.6708,72.8944,82.80,66,97,3.24,overcast clouds
17,17,Kapaa,US,2022-03-14 03:36:55,22.0752,-159.3190,80.58,70,20,10.36,few clouds
19,19,Banda Aceh,ID,2022-03-14 03:36:56,5.5577,95.3222,84.09,72,90,8.93,overcast clouds
21,21,Souillac,MU,2022-03-14 03:36:56,-20.5167,57.5167,79.18,89,75,8.05,light rain
25,25,Rikitea,PF,2022-03-14 03:36:57,-23.1203,-134.9692,80.49,71,70,14.36,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
#preferred_cities_df.head(10)
preferred_cities_df.count()


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Atuona,PF,79.03,overcast clouds,-9.8000,-139.0333,
2,Mahebourg,MU,81.25,broken clouds,-20.4081,57.7000,
4,Itarema,BR,77.27,overcast clouds,-2.9248,-39.9167,
7,Busselton,AU,78.85,broken clouds,-33.6500,115.3333,
9,Puerto Ayora,EC,76.55,scattered clouds,-0.7393,-90.3518,
11,Kudahuvadhoo,MV,82.80,overcast clouds,2.6708,72.8944,
17,Kapaa,US,80.58,few clouds,22.0752,-159.3190,
19,Banda Aceh,ID,84.09,overcast clouds,5.5577,95.3222,
21,Souillac,MU,79.18,light rain,-20.5167,57.5167,
25,Rikitea,PF,80.49,broken clouds,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Looking for rows with no Hotel Name
hotel_df.tail(20)

# index 644 has no name


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
598,Mont-Dore,NC,84.00,light rain,-22.2833,166.5833,GIte
600,Sorong,ID,83.93,broken clouds,-0.8833,131.2500,Waigo Splash Hotel
602,Puerto Escondido,MX,79.63,few clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
604,Palu,ID,88.63,scattered clouds,-0.8917,119.8707,Hotel Santika Palu
623,Ternate,ID,84.78,few clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
626,Tuy Hoa,VN,82.72,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
627,Cayhagan,PH,85.17,overcast clouds,9.6697,122.4256,Melrose Beach Resort
628,Maragogi,BR,78.08,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
637,Pringsewu,ID,85.59,overcast clouds,-5.3582,104.9744,Urban Style by Front One
638,Saint-Pierre,RE,77.92,light rain,-21.3393,55.4781,Lindsey Hôtel


In [11]:
# Count Rows
hotel_df.count()

City                   183
Country                183
Max Temp               183
Weather Description    183
Lat                    183
Lng                    183
Hotel Name             183
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.loc[hotel_df["Hotel Name"] != ""]  #if Hotel Name not equal to ""
clean_hotel_df.count()


City                   177
Country                177
Max Temp               177
Weather Description    177
Lat                    177
Lng                    177
Hotel Name             177
dtype: int64

In [13]:
#Verify hotels with no names are removed
clean_hotel_df.tail(20)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
597,Sur,OM,83.61,few clouds,22.5667,59.5289,Sur Plaza Hotel
598,Mont-Dore,NC,84.00,light rain,-22.2833,166.5833,GIte
600,Sorong,ID,83.93,broken clouds,-0.8833,131.2500,Waigo Splash Hotel
602,Puerto Escondido,MX,79.63,few clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
604,Palu,ID,88.63,scattered clouds,-0.8917,119.8707,Hotel Santika Palu
623,Ternate,ID,84.78,few clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
626,Tuy Hoa,VN,82.72,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
627,Cayhagan,PH,85.17,overcast clouds,9.6697,122.4256,Melrose Beach Resort
628,Maragogi,BR,78.08,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
637,Pringsewu,ID,85.59,overcast clouds,-5.3582,104.9744,Urban Style by Front One


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))